All the required libraries are imported.

In [2]:
import numpy as np
import pandas as pd
import urllib
import math
import statistics as st
from sklearn import preprocessing

import requests
from pandas import json_normalize
import json

from geopy.geocoders import Nominatim

import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans

The name of the town/city and the type of business or recreational venue is taken as input. The user can select a business type and has a list of options to choose from.

In [3]:
entry=input("Enter name of city/town along with state or country : ")

Enter name of city/town along with state or country : California


In [5]:
categories=pd.read_csv('/content/categories.csv')
categories=categories.sort_values('categories')

In [ ]:
for i in categories['categories'].values:
    print(i)

Adult Boutique
Afghan Restaurant
African Restaurant
American Restaurant
Amphitheater
Andhra Restaurant
Aquarium
Arcade
Argentinian Restaurant
Art Gallery
Arts & Crafts Store
Arts & Entertainment
Asian Restaurant
Athletics & Sports
Awadhi Restaurant
BBQ Joint
Bagel Shop
Bakery
Bar
Baseball Stadium
Basketball Stadium
Bath House
Beach
Bed & Breakfast
Beer Bar
Beer Store
Beijing Restaurant
Bengali Restaurant
Big Box Store
Bike Shop
Bistro
Boat or Ferry
Bookstore
Boutique
Brazilian Restaurant
Breakfast Spot
Brewery
Bridge
Bubble Tea Shop
Buddhist Temple
Building
Burger Joint
Butcher
Cafeteria
Café
Candy Store
Cantonese Restaurant
Capitol Building
Chaat Place
Cheese Shop
Chinese Restaurant
Chocolate Shop
Church
Clothing Store
Club House
Cocktail Bar
Coffee Shop
Comic Shop
Concert Hall
Construction & Landscaping
Convenience Store
Convention Center
Cosmetics Shop
Cricket Ground
Cultural Center
Cupcake Shop
Cycle Studio
Dance Studio
Deli , Bodega
Dentist's Office
Department Store
Dessert Shop
D

In [6]:
business=input('Select the type of business you are currently planning to open. Choose from the list given above : ')

Select the type of business you are currently planning to open. Choose from the list given above : Hostel


The required parameters are arranged to make a Foursquare API call.

In [7]:
address = entry

geolocator = Nominatim(user_agent="coursera-capstone-project")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [19]:
CLIENT_ID = 'SWWNHHEKWCPEHJKBPCWLGGN5BNKHHKU243TGGEY3MHO4ZK4H'
CLIENT_SECRET = '4RUFRQHM00UUVURJIIMO4ITB2V12C1NX1Z3KTWTORWW0AB5Z'
VERSION = '20180604'
LIMIT = 100
radius=5000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    latitude,
    longitude,
    radius,
    LIMIT)
results = requests.get(url).json()
print(results)

{'meta': {'code': 200, 'requestId': '670725da4971255ca01d5aa3'}, 'response': {'queryRefinements': {'target': {'type': 'path', 'url': '/venue/explore', 'params': {'ll': '36.701463,-118.755997', 'radius': '5000'}}, 'refinements': [{'query': 'Food'}, {'query': 'Nightlife'}, {'query': 'Coffee'}, {'query': 'Shops'}, {'query': 'Arts'}, {'query': 'Outdoors'}]}, 'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."}, 'headerLocation': 'Current map view', 'headerFullLocation': 'Current map view', 'headerLocationGranularity': 'unknown', 'totalResults': 3, 'suggestedBounds': {'ne': {'lat': 36.74646314500004, 'lng': -118.69997524373117}, 'sw': {'lat': 36.65646305499995, 'lng': -118.81201875626881}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'

The call returns a json file which is stored in a variable.

Only the 'items' are taken from the json file.

In [20]:
items = results['response']['groups'][0]['items']

The json file is normalized and converted into a dataframe.

In [ ]:
dataframe = json_normalize(items)
dataframe.head()

,reasons.count,reasons.items,referralId,venue.categories,venue.delivery.id,venue.delivery.provider.icon.name,venue.delivery.provider.icon.prefix,venue.delivery.provider.icon.sizes,venue.delivery.provider.name,venue.delivery.url,...,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.neighborhood,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups,venue.venuePage.id
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4bbf84aff8219c744d1db010-0,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",NaN,NaN,NaN,NaN,NaN,NaN,...,"[{'label': 'display', 'lat': 36.73520975345603...",36.735210,-119.731683,NaN,93727,CA,Los Panchos,0,[],NaN
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b771938f964a520387d2ee3-1,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,...,"[{'label': 'display', 'lat': 36.73646547723637...",36.736465,-119.694362,NaN,93727,CA,Sunnyside Deli,0,[],NaN
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4dcda119b0fb25f6e32f44a2-2,"[{'id': '4bf58dd8d48988d121941735', 'name': 'L...",NaN,NaN,NaN,NaN,NaN,NaN,...,"[{'label': 'display', 'lat': 36.73655874130699...",36.736559,-119.694835,NaN,93727,CA,O'Sullivan's Sunnyside Lounge,0,[],NaN
3,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4bd4b3356f64952196f16dec-3,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",1682806,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/wendys-765-...,...,"[{'label': 'display', 'lat': 36.7350852, 'lng'...",36.735085,-119.699654,NaN,93727,CA,Wendy’s,0,[],NaN
4,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b91b9ecf964a520f4d233e3-4,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,...,"[{'label': 'display', 'lat': 36.73672804808641...",36.736728,-119.698916,NaN,93727,CA,Sunrise Kitchen,0,[],NaN


The required columns are retrieved and renamed as required.

In [21]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']

    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [22]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues['categories']=nearby_venues['categories'].str.replace('/',',')

old_nearby_venues=nearby_venues

In [23]:
check_list=['Restaurant','Store','Shop','Garden','Park','Gym','Museum','Theatre','Hotel','Cafe','Market','Pub']
new_check_list=['Restaurant','Everyday Store','Everyday Shop','Garden','Park','Gym','Museum','Theatre','Hotel','Cafe','Market','Pub']

Data cleaning is thoroughly performed. Some venue categories are renamed to some convenient titles which would further make it easy and effective when the machine learning algorithms are applied. Two copies of the same dataframe is made but with slight variations. The stores,shops,bars and restaurants are not renamed for the dataframe 'old_nearby_venues' so as to enable the user in giving an accurate input later.

In [24]:
new=nearby_venues[~nearby_venues['categories'].str.contains('Restaurant')]
new=new[~new['categories'].str.contains('Bar')]
new=new[~new['categories'].str.contains('Plaza')]
new=new[~new['categories'].str.contains('Store')]
new=new[~new['categories'].str.contains('store')]
new=new[~new['categories'].str.contains('Shop')]
new=new[~new['categories'].str.contains('shop')]
new=new[~new['categories'].str.contains('Garden')]
new=new[~new['categories'].str.contains('Park')]
new=new[~new['categories'].str.contains('Gym')]
new=new[~new['categories'].str.contains('Museum')]
new=new[~new['categories'].str.contains('Theatre')]
new=new[~new['categories'].str.contains('Theater')]
new=new[~new['categories'].str.contains('Hotel')]
new=new[~new['categories'].str.contains('Café')]
new=new[~new['categories'].str.contains('Market')]
new=new[~new['categories'].str.contains('market')]
new=new[~new['categories'].str.contains('Pub')]
new=new[~new['categories'].str.contains('pub')]

In [25]:
res=nearby_venues[nearby_venues['categories'].str.contains('Restaurant')]
bar=nearby_venues[nearby_venues['categories'].str.contains('Bar')]
plaza=nearby_venues[nearby_venues['categories'].str.contains('Plaza')]
store=nearby_venues[nearby_venues['categories'].str.contains('Store')]
store1=nearby_venues[nearby_venues['categories'].str.contains('store')]
shop=nearby_venues[nearby_venues['categories'].str.contains('Shop')]
shop1=nearby_venues[nearby_venues['categories'].str.contains('shop')]
gar=nearby_venues[nearby_venues['categories'].str.contains('Garden')]
park=nearby_venues[nearby_venues['categories'].str.contains('Park')]
gym=nearby_venues[nearby_venues['categories'].str.contains('Gym')]
mus=nearby_venues[nearby_venues['categories'].str.contains('Museum')]
thr=nearby_venues[nearby_venues['categories'].str.contains('Theatre')]
thr1=nearby_venues[nearby_venues['categories'].str.contains('Theater')]
hot=nearby_venues[nearby_venues['categories'].str.contains('Hotel')]
cafe=nearby_venues[nearby_venues['categories'].str.contains('Café')]
mark=nearby_venues[nearby_venues['categories'].str.contains('Market')]
mark1=nearby_venues[nearby_venues['categories'].str.contains('market')]
pub=nearby_venues[nearby_venues['categories'].str.contains('Pub')]
pub1=nearby_venues[nearby_venues['categories'].str.contains('pub')]

res.categories='Restaurant'
bar.categories='Bar'
plaza.categories='Plaza'
store.categories='Everyday Store'
store1.categories='Everyday Store'
shop.categories='Everyday Shop'
shop1.categories='Everyday Shop'
gar.categories='Garden'
park.categories='Park'
gym.categories='Gym'
mus.categories='Museum'
thr.categories='Theatre'
thr1.categories='Theatre'
hot.categories='Hotel'
cafe.categories='Cafe'
mark.categories='Market'
mark1.categories='Market'
pub.categories='Pub'
pub1.categories='Pub'

The renamed column data is appended to the new dataframe named 'new' and is finally stored in the existing dataframe 'nearby_venues'.

In [27]:
import pandas as pd
new = pd.concat([new, res, bar, plaza, store, store1, shop, shop1, gar, park, gym, mus, thr, thr1, hot, cafe, mark, mark1, pub, pub1])

In [28]:
nearby_venues=new

In [29]:
new=old_nearby_venues[~old_nearby_venues['categories'].str.contains('Garden')]
new=new[~new['categories'].str.contains('Plaza')]
new=new[~new['categories'].str.contains('Park')]
new=new[~new['categories'].str.contains('Gym')]
new=new[~new['categories'].str.contains('Museum')]
new=new[~new['categories'].str.contains('Theatre')]
new=new[~new['categories'].str.contains('Theater')]
new=new[~new['categories'].str.contains('Hotel')]
new=new[~new['categories'].str.contains('Café')]
new=new[~new['categories'].str.contains('Market')]
new=new[~new['categories'].str.contains('market')]
new=new[~new['categories'].str.contains('Pub')]
new=new[~new['categories'].str.contains('pub')]

In [30]:
gar=old_nearby_venues[old_nearby_venues['categories'].str.contains('Garden')]
plaza=old_nearby_venues[old_nearby_venues['categories'].str.contains('Plaza')]
park=old_nearby_venues[old_nearby_venues['categories'].str.contains('Park')]
gym=old_nearby_venues[old_nearby_venues['categories'].str.contains('Gym')]
mus=old_nearby_venues[old_nearby_venues['categories'].str.contains('Museum')]
thr=old_nearby_venues[old_nearby_venues['categories'].str.contains('Theatre')]
thr1=old_nearby_venues[old_nearby_venues['categories'].str.contains('Theater')]
hot=old_nearby_venues[old_nearby_venues['categories'].str.contains('Hotel')]
cafe=old_nearby_venues[old_nearby_venues['categories'].str.contains('Café')]
mark=old_nearby_venues[old_nearby_venues['categories'].str.contains('Market')]
mark1=old_nearby_venues[old_nearby_venues['categories'].str.contains('market')]
pub=old_nearby_venues[old_nearby_venues['categories'].str.contains('Pub')]
pub1=old_nearby_venues[old_nearby_venues['categories'].str.contains('pub')]

In [31]:
gar.categories='Garden'
plaza.categories='Plaza'
park.categories='Park'
gym.categories='Gym'
mus.categories='Museum'
thr.categories='Theatre'
thr1.categories='Theatre'
hot.categories='Hotel'
cafe.categories='Café'
mark.categories='Market'
mark1.categories='Market'
pub.categories='Pub'
pub1.categories='Pub'

The renamed column data is appended to the new dataframe named 'new' and is finally stored in the existing dataframe 'old_nearby_venues'.

In [33]:
import pandas as pd

new = pd.concat([new, gar, plaza, park, gym, mus, thr, thr1, hot, cafe, mark, mark1, pub, pub1])

In [34]:
old_nearby_venues=new

The 'categories' column data is split into two strings with the delimiter being comma(,), and only the first string is kept.

In [35]:
old_nearby_venues['categories'] = old_nearby_venues['categories'].str.split(',').str[0]

In [36]:
nearby_venues['categories'] = nearby_venues['categories'].str.split(',').str[0]

White spaces from the ends of string are removed.

In [37]:
old_nearby_venues.categories = old_nearby_venues.categories.str.rstrip()

In [38]:
row_count = nearby_venues.shape[0]
row_count

3

One hot encoding is performed on the 'categories' column.

In [39]:
onehot = pd.get_dummies(nearby_venues[['categories']], prefix="", prefix_sep="")

The 'sum of squared mean' is run 20 times to get the maximum optimum value of k.

In [40]:
n=20
ks=np.zeros((n-1))
max_k_optimum=0

for count in range(1,n):
    if row_count < 11:
        k_rng=range(1,row_count)
        sse=[]
        for k in k_rng:
            km=KMeans(n_clusters=k)
            km.fit(onehot)
            sse.append(km.inertia_)

        a=sse[0]-sse[row_count-2]
        b=k_rng[row_count-2]-k_rng[0]
        c1=k_rng[0] * sse[row_count-2]
        c2=k_rng[row_count-2] * sse[0]
        c= c1 - c2

        def calc_distance(x1, y1, a, b, c):
            d=abs((a * x1 + b * y1 + c)) / (math.sqrt(a * a + b * b))
            return d

        points=[]
        for k in range(row_count-1):
            points.append(calc_distance(k_rng[k], sse[k], a, b, c))

        k_optimum=str(points.index(max(points))+1)
        k_optimum=(int)(k_optimum)
        ks[count-1]=k_optimum
        if ks[count-1] > max_k_optimum:
            max_k_optimum=ks[count-1]

    else:
        k_rng=range(1,25)
        sse=[]
        for k in k_rng:
            km=KMeans(n_clusters=k)
            km.fit(onehot)
            sse.append(km.inertia_)

        a=sse[0]-sse[23]
        b=k_rng[23]-k_rng[0]
        c1=k_rng[0] * sse[23]
        c2=k_rng[23] * sse[0]
        c= c1 - c2

        def calc_distance(x1, y1, a, b, c):
            d=abs((a * x1 + b * y1 + c)) / (math.sqrt(a * a + b * b))
            return d

        points=[]
        for k in range(24):
            points.append(calc_distance(k_rng[k], sse[k], a, b, c))

        k_optimum=str(points.index(max(points))+1)
        k_optimum=(int)(k_optimum)
        ks[count-1]=k_optimum
        if ks[count-1] > max_k_optimum:
            max_k_optimum=ks[count-1]

In [41]:
max_k_optimum=(int)(max_k_optimum)
print(max_k_optimum)

1


The k-means clustering algorithm is applied on the one hot encoded data with the value of k being the maximum optimum value of k calculated from the above piece of code.

In [42]:
kmeans=KMeans(n_clusters=max_k_optimum,random_state=0).fit(onehot)

The cluster labels are inserted into the dataframe 'nearby_venues' in a column named 'cluster'.

In [43]:
nearby_venues.insert(0, 'cluster', kmeans.labels_)

A folium map is generated displaying all the venue categories clustered in different colors.

In [44]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(max_k_optimum)
ys = [i + x + (i*x)**2 for i in range(max_k_optimum)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, name, cluster in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name'], nearby_venues['cluster']):
    label = folium.Popup(' Cluster ' + str(cluster) + ':  ' + str(name), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

The major venue categories along with their percentages are taken into two different lists. Also, the average latitudes and longitudes of the major categories are stored in other lists.

In [45]:
avg_lat=[]
avg_lng=[]
clustered_name=[]
clustered_percent=[]
identifier=0
final=0

for i in range(0,max_k_optimum):
    cust=nearby_venues[nearby_venues['cluster']==i]
    cust=pd.DataFrame(cust)
    if len(cust['categories'].unique()) == 1:
        count=cust.shape[0]
        mean_lat=cust['lat'].mean()
        mean_lng=cust['lng'].mean()
        avg_lat.append(mean_lat)
        avg_lng.append(mean_lng)
        val=cust['categories'].values[0]
        n=len(cust)
        percent=(n/row_count)*100
        percent=round(percent)
        clustered_name.append(val)
        clustered_percent.append(percent)
    else:
        n=len(cust)
        percent=(n/row_count)*100
        percent=round(percent)
        val='Miscellaneous'
        final=1
        clustered_name.append(val)
        clustered_percent.append(percent)

The number of venue categories the user would love to have around his/her business is taken as input. Also, the venue categories of that particular town/city is displayed for the user to make a choice from.

In [49]:
options=[]
c=0
df=pd.DataFrame(columns=['categories'])
df.categories=old_nearby_venues['categories']
df=df.categories.unique()
df=pd.DataFrame(df,columns=['categories'])
df=df.sort_values('categories')
for i in df['categories'].values:
    print(i)
    c=c+1
print()
maximum=input("Enter number of venues you would love to have around your business from the list above. You can enter a maximum of " + (str)(c) + " venues. ")

Campground
Lake
Mountain

Enter number of venues you would love to have around your business from the list above. You can enter a maximum of 3 venues. 3


In [50]:
for i in range(0,(int)(maximum)):
    option=input('Option ' + (str)(i+1) + ' : ')
    options.append(option)

Option 1 : museum
Option 2 : airport
Option 3 : hotel


The new possible coordinates are calculated based on the information given by the user.

In [56]:
opt_lat = []
opt_lng = []
lat = 0
lng = 0
count = 0

for i in range(len(options)):
    check = options[i]
    for (j, k, l) in zip(old_nearby_venues['categories'].values, old_nearby_venues['lat'].values, old_nearby_venues['lng'].values):
        if j == check:
            count += 1
            lat += k
            lng += l

    # Check to avoid division by zero
    if count > 0:
        lat = lat / count
        lng = lng / count
    else:
        lat, lng = None, None  # or use some default values if no match is found

    opt_lat.append(lat)
    opt_lng.append(lng)

    # Reset values for the next iteration
    lat = 0
    lng = 0
    count = 0


In [55]:

import statistics as st

# Filter out None values before calculating the mean
valid_opt_lat = [x for x in opt_lat if x is not None]
valid_opt_lng = [x for x in opt_lng if x is not None]

# Now calculate the mean
if valid_opt_lat and valid_opt_lng:  # Ensure there are valid values to calculate the mean
    lat = st.mean(valid_opt_lat)
    lng = st.mean(valid_opt_lng)
else:
    lat, lng = None, None  # Handle the case where there are no valid values

In [57]:
result=-1
j=0
for i in check_list:
    result=business.find(i)
    if result!=-1:
        business=new_check_list[j]
    j=j+1

The support vector machine classification algorithm is applied to predict the category of the business or venue given as input by the user.

In [58]:
if len(clustered_name) > 1:
    un1=''
    un2=''
    row = [(un1,un2,business,lat,lng)]

    row = pd.DataFrame(row, columns = ['cluster', 'name', 'categories', 'lat', 'lng'])

    feature = pd.DataFrame()
    feature = pd.concat([feature,pd.get_dummies(nearby_venues['categories'])], axis=1)
    x = preprocessing.StandardScaler().fit(feature).transform(feature)
    y = nearby_venues['cluster']

    from sklearn.model_selection import train_test_split
    x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=0.0, random_state=4)

    from sklearn import svm
    SVM = svm.SVC()
    SVM.fit(x_train, y_train)

    flag=0
    predicted_cluster=-1
    for i in nearby_venues['categories'].values:
        if i==business:
            flag=1
    if flag==1:
        nearby_venues=nearby_venues.append(row,ignore_index=True)
        feature = pd.DataFrame()
        feature = pd.concat([feature,pd.get_dummies(nearby_venues['categories'])], axis=1)
        x = preprocessing.StandardScaler().fit(feature).transform(feature)
        svm_predict = SVM.predict(x)
        array_length = len(svm_predict)
        last_element = svm_predict[array_length - 1]
        predicted_cluster=(int)(last_element)
    if final==1 and flag==0:
        predicted_cluster=0

Required markers are added to the folium map i.e the new coordinates.

In [59]:
folium.Marker(
    location=[lat,lng],
    icon=folium.Icon(icon='cloud')
).add_to(map_clusters)

The result is a folium map with the popup being the required coordinates of the location where the business or the venue could be possibly set up, along with the following information derived from various input taken from the user.

In [60]:
print('Major categories in ' + entry + ' are -->')
print()
for (i,j) in (zip(clustered_name,clustered_percent)):
    print((str)(i) + ' : ' + (str)(j) + '%')
print()
if len(clustered_name) > 1:
    if flag==0 and final==0:
        print('Your business does not fall into any category we have made. This is a new type of business you are trying to open which is almost not present in the location you have provided. This significantly boosts the chances of you being successful in your business. Based on the data you have provided to us, we recommend you to set up your business within 1 km radius of the popup marked on the map given below.')
    else:
        print('The kind of business you are trying to open falls into the ' + (str)(clustered_percent[predicted_cluster]) + '% ' + (str)(clustered_name[predicted_cluster]) + ' category. Based on the data you have provided to us, we recommend you to set up your business within 1 km radius of the popup marked on the map given below.')
    print('Popup coordinates --> Latitude : ' + (str)(lat) + '  Longitude : ' + (str)(lng))
if len(clustered_name) == 1:
    print('The kind of business you are trying to open falls into the one and only ' + (str)(clustered_name[0]) + ' category. Based on the data you have provided to us, we recommend you to set up your business within 1 km radius of the popup marked on the map given below.')
    print('Popup coordinates --> Latitude : ' + (str)(lat) + '  Longitude : ' + (str)(lng))
map_clusters

Major categories in California are -->

Miscellaneous : 100%

The kind of business you are trying to open falls into the one and only Miscellaneous category. Based on the data you have provided to us, we recommend you to set up your business within 1 km radius of the popup marked on the map given below.
Popup coordinates --> Latitude : 0  Longitude : 0
